In [9]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from huggingface_hub import hf_hub_download
import shutil
import os
from langchain_core.tools import tool
from langchain.tools.render import render_text_description_and_args

In [14]:
!pip3 list |grep llama

llama_cpp_python          0.2.76


In [5]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int
@tool
def korea_city_weather_info(city_name: str) -> str:
    """
    Tool Description : The weather information of the city inputted by the user is outputted.
    Args : 
        "city_name" : Name of the city to look up the weather
    """
    return "오늘 서울의 온도는 최고 24.2도, 최저 13.5도를 보이며, 낮 한때 소나기가 있겠습니다."

@tool
def search_phonenumber(query: str) -> str:
    """장소에 대한 전화번호 검색 결과를 반환할 때 사용되는 도구입니다. 전화번호를 알고싶을때 사용하는 도구입니다."""
    return "판교 몽중헌 전화번호: 010-1234-5678\n\n서울 OOO 전화번호: 02-123-4567"

@tool
def get_date():
    """오늘의 날짜를 알고싶을때, 사용하는 도구입니다."""
    from datetime import datetime
    date = datetime.now()
    return f"오늘은 {date.year}년{date.month}월{date.day}일 입니다."

@tool
def get_team_member(team_name:str):
    """
    팀의 구성원 정보를 조회하는 도구입니다.
    <Argument>
    - team_name : 구성원을 조회할 팀명을 입력합니다.
    <질문예시> 
        - 데이터시각화팀 인원보여줘
        - 설계팀 구성원 보여줘
        - 기획4팀 팀원이 누가있지?
    """
    return '손철준, 김민지D, 이재엽, 김원태, 이흥배, 허재우, 오형록, 신유나, 이승연C, 김상웅, 이창윤, 양성렬 총 12명입니다. '


@tool
def animal_habitat_search(animal_name:str):
    """
    동물의 주요 서식지 정보를 알고싶을때 사용하는 도구입니다.
    """
    return f'{animal_name}의 주요서식지 : 아마존 강남구 31번지'


@tool
def working_time_search(employee_nm:str):
    """
    조회한 직원의 근무시간을 알고싶을때 사용하는 도구입니다.
    <Argument>
    - employee_nm : 근무시간을 조회할 직원의 이름을 입력합니다.
    """
    return f'{employee_nm}의 {datetime.now().year}년 {datetime.now().month}월 총 근무시간 : 999시간'

tools = [korea_city_weather_info, search_phonenumber, multiply, get_date, get_team_member, animal_habitat_search, working_time_search]

In [6]:
# !CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python
# !huggingface-cli download Bllossom/llama-3-Korean-Bllossom-70B-gguf-Q4_K_M --local-dir='YOUR-LOCAL-FOLDER-PATH'

from llama_cpp import Llama
from transformers import AutoTokenizer

model_id = 'Bllossom/llama-3-Korean-Bllossom-70B-gguf-Q4_K_M'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Llama(
    model_path='/workspace/wontae_kim/llama-3-Korean-Bllossom-70B-gguf-Q4_K_M.gguf',
    n_ctx=1024,
    n_gpu_layers=-1        # Number of model layers to offload to GPU
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 22 key-value pairs and 723 tensors from /workspace/wontae_kim/llama-3-Korean-Bllossom-70B-gguf-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hslim
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 145088
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 8192
llama_model_loader: - kv   5:                          llama.block_count u32              = 80
llama_model_loader: - kv

In [7]:
def tool_arg_generator(user_input:str, tools):
    print(f'\n--> User Question : {user_input}\n\n')
    tools_info = render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')
    tools_name = [t.name for t in tools]
    
    query = f"{user_input} --> 라는 질문과 관련된 tool_name과 Argument에 대해서 system_prompt를 참고한 양식으로 출력해줘"
    
    system_prompt = f"""
    당신은 주어진 질문에 적절한 도구(Tool)을 선택하여 제시하는 역할을 합니다..
    You are responsible for selecting and presenting the appropriate tool for the given question.
    
    * 사용자 질문(Question)에 대해 아래 'instructions'와 '출력예시'을 참고해서 사용할 Tool과 Tool에 input할 argument를 출력해라.
    
    <Instructions>
    - 아래 Tool List의 도구 중, 사용자의 질문과 관련된 도구를 선택하고 도구 이름과 도구에 입력해야할 인수(args)를 아래와 같은 Json형태의 데이터로 출력해라
    - 반드시 아래의 Example of output의 양식을 지켜 반드시 Json 형태로 출력하라
    - Tool name은 반드시 'Tools Name'에 포함된 이름이어야한다.
    - 'action'과 'action_input'에 포함될 key에는 공백이 절대로 들어갈 수 없다.
    
    <Tool List>
    {tools_info}
    
    <Tools Name>
    {tools_name}
    
    <출력예시>
    {{
        "action": '사용할 도구의 이름'
        "action_input": '도구에 input되어야 할 argument를 전달합니다.'
    }}

    <출력예시 관련 지시사항>
    - 'action'에 포함되는 도구이름에는 반드시 공백을 제거한다.
    - 'action_input'에 포함되는 dictionary에는 type정보는 표시하지 않는다.
    - 'action_input'에 포함되는 dictionary의 key는 반드시 공백을 제거한다.
    """
    
    messages = [
        {"role": "system", "content": f"{system_prompt}"},
        {"role": "user", "content": f"{query}"}
        ]
    
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize = False,
        add_generation_prompt=True
    )
    
    generation_kwargs = {
        "max_tokens":1024,
        "stop":["<|eot_id|>"],
        "echo":True, # Echo the prompt in the output
        "top_p":0.9,
        "temperature":0.001,
    }
    
    resonse_msg = model(prompt, **generation_kwargs)
    get_tool_info = resonse_msg['choices'][0]['text'][len(prompt):]
    print(f'************* 모델을 통해 질문에 적합한 Tool 선택 및 Argument생성을 완료하였습니다.*************')
    print(f'{get_tool_info}\n ********************************************')
    print(get_tool_info)
    get_tool_info = get_tool_info.lower()
    return get_tool_info


def tool_executor_parser(tool_description):
    stop_key = ['Type', 'string']
    des =f"""
    Action:```
    {tool_description}
    ```
    """
    from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
    parser = ReActJsonSingleInputOutputParser()
    action = parser.parse(des)
    action.tool = action.tool.strip().replace('-', '_').replace(' ', '_').lower() # tool 이름 정리하자.
    action.tool_input = {k.strip().replace('-', '_').replace(' ', '_').lower():v for k, v in action.tool_input.items() if k not in stop_key} # tool 내부의 인수를 정리하자.
    print(f'\n\n1) Tool_Name : {action.tool} \n2) Tool Input Argument : {action.tool_input}\n********************************************')
    return action


def cutom_Tool_Executor(user_input, tools):
    """
    * tools : 함수에 '@'데코레이터를 붙인 후 리스트에 담은 형태 필요
    * action : 내부에 'Action' 키워드로 감싼 후, action, action_input이 포함된 dictionary를 'ReActJsonSingleInputOutputParser'클래스로 파싱처리한 데이터 input필요
    """
    tool_info = tool_arg_generator(user_input=user_input, tools=tools)
    action = tool_executor_parser(tool_info)
    
    from langgraph.prebuilt import ToolExecutor
    tool_executor = ToolExecutor(tools)
    print('\n\n\n======================== TOOL RETURN ========================')
    return tool_executor.invoke(action)

In [10]:
# user_input = "서울의 날씨 알려줘"
# user_input = '판교 몽중헌의 전화번호 알려줘'
# user_input = '오늘 날짜 알려줘'
user_input = '데이터사이언스셀의 팀원정보 보여줘'
# user_input = '판다는 보통 어디에 살지?'
# user_input = '이흥배 주임연구원의 근무시간알려줘'
cutom_Tool_Executor(user_input, tools)

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?



--> User Question : 데이터사이언스셀의 팀원정보 보여줘





llama_print_timings:        load time =    2465.52 ms
llama_print_timings:      sample time =      55.90 ms /    38 runs   (    1.47 ms per token,   679.75 tokens per second)
llama_print_timings: prompt eval time =    4512.68 ms /   924 tokens (    4.88 ms per token,   204.76 tokens per second)
llama_print_timings:        eval time =    1793.84 ms /    37 runs   (   48.48 ms per token,    20.63 tokens per second)
llama_print_timings:       total time =    6418.91 ms /   961 tokens


************* 모델을 통해 질문에 적합한 Tool 선택 및 Argument생성을 완료하였습니다.*************
{  
  "action": "get-team-member",  
  "action_input": {  
    "Team Name": "데이터사이언스셀"  
  }  
}
 ********************************************
{  
  "action": "get-team-member",  
  "action_input": {  
    "Team Name": "데이터사이언스셀"  
  }  
}


1) Tool_Name : get_team_member 
2) Tool Input Argument : {'team_name': '데이터사이언스셀'}
********************************************



======================== TOOL RETURN ========================


'손철준, 김민지D, 이재엽, 김원태, 이흥배, 허재우, 오형록, 신유나, 이승연C, 김상웅, 이창윤, 양성렬 총 12명입니다. '